In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from googleapiclient.discovery import build


In [ ]:
# Set up API key and YouTube service
api_key = 'YOUTUBE_Api_Key'
youtube = build('youtube', 'v3', developerKey=api_key)


##function to get stats

In [ ]:
def get_channel_data(channel_id):
    
    request = youtube.channels().list(
        part='snippet,contentDetails,statistics',
        id=channel_id
    )
    response = request.execute()
    return response['items'][0]

def get_video_data(channel_id):
    
    request = youtube.search().list(
        part='snippet',
        channelId=channel_id,
        maxResults=50,
        order='date'
    )
    response = request.execute()
    return response['items']

def extract_video_details(videos):
   
    video_details = []
    for video in videos:
        # Ensure 'videoId' is present in the response
        if 'id' in video and 'videoId' in video['id']:
            video_id = video['id']['videoId']
            snippet = video['snippet']
            video_details.append({
                'video_id': video_id,
                'title': snippet['title'],
                'published_at': snippet['publishedAt']
            })
        else:
            print(f"Video ID not found in response: {video}")
    return video_details


In [ ]:
# Replace with your Kenyan YouTube channel IDs
channel_ids = ['UC33Tc4FCgd_to314_b2hNTg', 'UCiuh2rVpl8I_wZQASC6wxbg']

# Fetch and analyze data
channel_data_list = []
for channel_id in channel_ids:
    channel_data = get_channel_data(channel_id)
    video_data = get_video_data(channel_id)
    video_details = extract_video_details(video_data)
    
    # Collect channel data
    channel_data_list.append({
        'channel_id': channel_id,
        'channel_title': channel_data['snippet']['title'],
        'subscriber_count': channel_data['statistics']['subscriberCount'],
        'video_count': channel_data['statistics']['videoCount'],
        'view_count': channel_data['statistics']['viewCount'],
        'latest_videos': video_details
    })

# Create DataFrame for analysis
df = pd.DataFrame(channel_data_list)


In [ ]:

# Display the DataFrame
print(df[['channel_title', 'subscriber_count', 'video_count', 'view_count']])



In [ ]:

#  Plot subscriber counts
df.set_index('channel_title')['subscriber_count'].astype(int).plot(kind='bar')
plt.title('Subscriber Counts of Kenyan YouTube Channels')
plt.xlabel('Channel Title')
plt.ylabel('Subscriber Count')
plt.xticks(rotation=45)
plt.show()
